In [103]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import seaborn as sns
from scipy import optimize
import time
import functools
from multiprocessing import Pool
#from cyipopt import minimize_ipopt
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt
import random

import warnings
warnings.filterwarnings("ignore")

random.seed(10)


In [104]:
input_dir = r"/Users/juju/Library/Mobile Documents/com~apple~CloudDocs/ta_project/output/welfare/fixed_price"
input_dir_eq = r"/Users/juju/Library/Mobile Documents/com~apple~CloudDocs/ta_project/src"

In [105]:
data = pd.read_csv(os.path.join(input_dir,r"welfare_fixed_price_result_obs_less_brand_ch3.csv"))

data_eq = pd.read_csv(os.path.join(input_dir_eq,r"df_ne.csv"))

In [106]:
list(data_eq.columns)

['index',
 'post_month',
 'shareid',
 'year_month',
 'Supply',
 'Demand',
 'Revenue',
 'Occ',
 'ADR',
 'RevPAR',
 'Country',
 'Market',
 'Chain',
 'Owner',
 'Mgmt',
 'ParentCompany',
 'Operation',
 'Scale',
 'Class',
 'Location',
 'OpenDate',
 'SizeCode',
 'max_str_month',
 'max_review_month',
 'min_review_month',
 'on_ta',
 'monthly_rating_mean',
 'monthly_review_count',
 'monthly_one_star_count',
 'monthly_two_star_count',
 'monthly_three_star_count',
 'monthly_four_star_count',
 'monthly_five_star_count',
 'monthly_hotel_response_count',
 'monthly_partnership_reviews_count',
 'cleanliness_count',
 'cleanliness_mean',
 'location_count',
 'location_mean',
 'sleep_quality_count',
 'sleep_quality_mean',
 'room_quality_count',
 'room_quality_mean',
 'service_count',
 'service_mean',
 'value_count',
 'value_mean',
 'renovate',
 'solo_count',
 'couple_count',
 'family_count',
 'business_count',
 'month_with_reviews',
 'num_of_reviews',
 'total_one_star',
 'total_two_star',
 'total_three_st

In [107]:

alpha = -0.012
sigma = 0.5396
rating_param_chain = 0.075-0.016
rating_param_ind = 0.075

occ_upper= 88


In [108]:

data = data.merge(data_eq[["q_upper","hotel_fixed_effect","market_time_fixed_effect",\
                          "shareid","year","month","max_group_price","market_size"]], on=["shareid","year","month"])


In [109]:
supply_max = data.groupby(["shareid","year"])["Supply"].max().reset_index().rename(columns={"Supply":"max_supply_year"})
data= data.merge(supply_max, on=["shareid","year"], how="left")
data["capacity"] = data["max_supply_year"]/30

# Calculate marginal cost and total cost

In [110]:
data_lower = data[data["Demand"]<=data["q_upper"]]

data_upper = data[data["Demand"]>data["q_upper"]]

data_lower["excess_demand"] = 0
data_lower["cost_ne"] = (192.39814102368837+data_lower["hotel_fixed_effect"]+data_lower["market_time_fixed_effect"])*\
                           data_lower["Demand"]
data_upper["excess_demand"] = data_upper["Demand"]-data_upper["q_upper"]
data_upper["cost_ne"] = (192.39814102368837+data_upper["hotel_fixed_effect"]+data_upper["market_time_fixed_effect"])*\
                           data_upper["q_upper"]+0.5*0.043841100663268406*(data_upper["excess_demand"]**2)


data= pd.concat([data_lower,data_upper],axis=0)

data = data.fillna(0)



In [111]:
data

,level_0,post_month,shareid,year_month,Supply,Demand,Revenue,Occ,ADR,RevPAR,...,D_g_sum,q_upper,hotel_fixed_effect,market_time_fixed_effect,max_group_price,market_size,max_supply_year,capacity,excess_demand,cost_ne
0,769,2016-01,80022,2016-01-01,3658.0,2638.0,109250.0,72.1159,41.413950,29.866047,...,1.443469,3219.04,-183.251750,0.000000,143.847895,1419322.0,3658.0,121.933333,0.00,24128.180335
1,770,2016-02,80022,2016-02-01,3304.0,2561.0,97336.0,77.5121,38.007029,29.460048,...,1.796726,2907.52,-183.251750,4.713191,169.001597,1419322.0,3658.0,121.933333,0.00,35494.389401
2,771,2016-03,80022,2016-03-01,3658.0,3217.0,123244.0,87.9442,38.310227,33.691635,...,2.213311,3219.04,-183.251750,4.462434,221.553332,1419322.0,3658.0,121.933333,0.00,43779.590612
4,773,2016-05,80022,2016-05-01,3658.0,3102.0,121392.0,84.8004,39.133462,33.185347,...,1.885662,3219.04,-183.251750,6.055803,175.144899,1419322.0,3658.0,121.933333,0.00,47157.206564
5,774,2016-06,80022,2016-06-01,3540.0,2863.0,112845.0,80.8757,39.414949,31.877119,...,1.637039,3115.20,-183.251750,-2.745130,161.727004,1419322.0,3658.0,121.933333,0.00,18326.809963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5591,763,2016-07,8116969,2016-07-01,7874.0,7496.0,1303426.0,95.1993,173.882871,165.535433,...,6.719439,6929.12,-167.680053,69.794834,272.152682,1210161.0,7874.0,262.466667,566.88,661935.615278
5592,764,2016-08,8116969,2016-08-01,7874.0,7493.0,951448.0,95.1612,126.978246,120.834138,...,4.997755,6929.12,-167.680053,57.166410,251.211404,1210161.0,7874.0,262.466667,563.88,574357.389290
5593,765,2016-09,8116969,2016-09-01,7620.0,7098.0,1295898.0,93.1496,182.572274,170.065354,...,4.827411,6705.60,-167.680053,96.538286,294.127388,1210161.0,7874.0,262.466667,392.40,816472.021941
5594,766,2016-10,8116969,2016-10-01,7678.0,7501.0,1284279.0,97.6947,171.214371,167.267387,...,5.620897,6756.64,-167.680053,101.408458,307.181163,1210161.0,7874.0,262.466667,744.36,864337.229675


In [112]:
# Counterfactual marginal cost

data["demand_cf"] = data["market_share_cf"]*data["market_size"]

data_lower = data[data["demand_cf"]<=data["q_upper"]]
data_upper = data[data["demand_cf"]>data["q_upper"]]

data_lower["excess_demand"] = 0
data_lower["cost_cf"] = (192.39814102368837+data_lower["hotel_fixed_effect"]+data_lower["market_time_fixed_effect"])*\
                           data_lower["demand_cf"]
data_upper["excess_demand"] = data_upper["demand_cf"]-data_upper["q_upper"]
data_upper["cost_cf"] = (192.39814102368837+data_upper["hotel_fixed_effect"]+data_upper["market_time_fixed_effect"])*\
                           data_upper["q_upper"]+0.5*0.043841100663268406*(data_upper["excess_demand"]**2)

data = pd.concat([data_lower,data_upper],axis=0)



# Calculate Consumer Surplus

In [113]:
# calculate walfare

data_chain = data[data["operation"]=='Chain_affiliated'].reset_index().drop(["index"],axis=1)
data_ind = data[data["operation"]=='Independent'].reset_index().drop(["index"],axis=1)

data_chain["market_share_cf_delta_diff"] = data_chain["market_share_cf"]*rating_param_chain*(data_chain["Rating"] - data_chain["exante_rating"])
data_ind["market_share_cf_delta_diff"] = data_ind["market_share_cf"]*rating_param_ind*(data_ind["Rating"] - data_ind["exante_rating"])
 
    
data = pd.concat([data_chain, data_ind],axis=0).reset_index().drop(["index"],axis=1)

l=[]
for i in (0,1,2):
    
    df = data[data["Market"]==data["Market"].unique()[i]]
    diff = df.groupby(["month","year"])["market_share_cf_delta_diff"].sum().reset_index().rename(columns={"market_share_cf_delta_diff":"diff_term"})
    temp = df.merge(diff,on=["month","year"], how="left")
    l.append(temp)
    
df = pd.concat(l,axis=0)


In [114]:
df["r_greater_exante"] = df["exante_rating"]<df["Rating"]

#df["market_share_percent_diff"] = (df["market_share_cf"] - df["market_share"])/df["market_share"]


In [115]:
df_cs = df[["Market","year","month","Market Monthly Supply","D_g_sum","D_g_sum_cf","diff_term","market_size"]].drop_duplicates()

df_cs["cs"] = (np.log(1+df_cs["D_g_sum"]) - np.log(1+df_cs["D_g_sum_cf"])-df_cs["diff_term"])/alpha



In [116]:
df_cs["month_cs"] = df_cs["cs"]*df_cs["market_size"]

cs_result = df_cs.groupby("Market")["month_cs"].sum()/1000000
cs_result = cs_result.reset_index("Market")
cs_result.transpose()


,0,1,2
Market,"Chicago, IL","Houston, TX","Miami/Hialeah, FL"
month_cs,-0.213338,-0.402672,-0.1051


# Calculate Revenue, Profit, Marginal Cost

In [117]:
df["Revenue_cf"] = df["market_share_cf"]*df["market_size"]*df["ADR"]
df["revenue_diff"] = df["Revenue_cf"] - df["Revenue"]
revenue_result = df.groupby("Market")["revenue_diff"].sum()/1000000
revenue_result = revenue_result.reset_index("Market")
revenue_result.transpose()


,0,1,2
Market,"Chicago, IL","Houston, TX","Miami/Hialeah, FL"
revenue_diff,-3.136118,2.35796,12.681274


In [118]:
revenue_result["revenue_diff"].sum()

11.903116393097495

In [119]:
revenue_breakdown_result = df.groupby(["Market","r_greater_exante"])["revenue_diff"].sum()/1000000
revenue_breakdown_result.reset_index().transpose()                                      

,0,1,2,3,4,5
Market,"Chicago, IL","Chicago, IL","Houston, TX","Houston, TX","Miami/Hialeah, FL","Miami/Hialeah, FL"
r_greater_exante,False,True,False,True,False,True
revenue_diff,25.417559,-28.553677,19.669386,-17.311426,15.449122,-2.767848


In [120]:
df.groupby(["r_greater_exante"])["revenue_diff"].sum()/1000000

r_greater_exante
False    60.536067
True    -48.632951
Name: revenue_diff, dtype: float64

In [121]:
df["ps_cf"] = df["Revenue_cf"]-df["cost_cf"]
df["ps_ne"] = df["Revenue"]-df["cost_ne"]
df["ps_diff"] = df["ps_cf"] - df["ps_ne"]
df.groupby("Market")["ps_diff"].sum()/1000000

Market
Chicago, IL          2.139210
Houston, TX          1.776034
Miami/Hialeah, FL    5.041118
Name: ps_diff, dtype: float64

In [122]:
df["ps_diff"].sum()/1000000

8.956362091163829

In [123]:
df.groupby(["r_greater_exante"])["ps_diff"].sum()/1000000

r_greater_exante
False    24.546665
True    -15.590303
Name: ps_diff, dtype: float64

In [124]:
df["cost_diff"] = df["cost_cf"] - df["cost_ne"]
df["cost_diff"].sum()/1000000



2.9467543019336655

In [125]:
df.groupby(["r_greater_exante"])["cost_diff"].sum()/1000000

r_greater_exante
False    35.989403
True    -33.042648
Name: cost_diff, dtype: float64

In [126]:
df.groupby("Market")["cost_diff"].sum()/1000000

Market
Chicago, IL         -5.275328
Houston, TX          0.581926
Miami/Hialeah, FL    7.640156
Name: cost_diff, dtype: float64

In [127]:
total_market_share = df.groupby(["Market","year","month"])["market_share"].sum().reset_index().rename(columns={"market_share":"market_share_sum"})
total_market_share_cf = df.groupby(["Market","year","month"])["market_share_cf"].sum().reset_index().rename(columns={"market_share_cf":"market_share_sum_cf"})

df_2 = df.merge(total_market_share, on=["Market","year","month"]).merge(total_market_share_cf, on=["Market","year","month"])
                                                                                                           



In [128]:
df_2["market_share_sum_diff"] = df_2["market_share_sum_cf"] - df_2["market_share_sum"]

total_market_share = df_2[["Market","year","month","market_share_sum_diff","market_size"]].drop_duplicates()

total_market_share["market_share_sum_diff"].mean()*100

-0.003824352087221989

In [129]:
total_market_share = df.groupby(["Market","year","month"])["market_share"].sum().reset_index().rename(columns={"market_share":"market_share_sum"})
total_market_share_cf = df.groupby(["Market","year","month"])["market_share_cf"].sum().reset_index().rename(columns={"market_share_cf":"market_share_sum_cf"})

df_2 = df.merge(total_market_share, on=["Market","year","month"]).merge(total_market_share_cf, on=["Market","year","month"])
                                                                                                           


In [130]:
total_market_share = df.groupby(["operation","year","month"])["market_share"].sum().reset_index().rename(columns={"market_share":"market_share_sum"})
total_market_share_cf = df.groupby(["operation","year","month"])["market_share_cf"].sum().reset_index().rename(columns={"market_share_cf":"market_share_sum_cf"})

df_3 = df.merge(total_market_share, on=["operation","year","month"]).merge(total_market_share_cf, on=["operation","year","month"])

df_3["market_share_sum_diff"] = df_3["market_share_sum_cf"] - df_3["market_share_sum"]

total_market_share = df_3[["operation","year","month","market_share_sum_diff","market_size"]].drop_duplicates()

        

In [131]:
total_market_share.groupby("operation")["market_share_sum_diff"].mean()*100

operation
Chain_affiliated   -0.268088
Independent         0.256615
Name: market_share_sum_diff, dtype: float64

In [132]:
total_market_share = df.groupby(["operation","r_greater_exante","year","month"])["market_share"].sum().reset_index().rename(columns={"market_share":"market_share_sum"})
total_market_share_cf = df.groupby(["operation","r_greater_exante","year","month"])["market_share_cf"].sum().reset_index().rename(columns={"market_share_cf":"market_share_sum_cf"})

df_4 = df.merge(total_market_share, on=["operation","r_greater_exante","year","month"]).merge(total_market_share_cf, on=["operation","year","month","r_greater_exante"])

df_4["market_share_sum_diff"] = df_4["market_share_sum_cf"] - df_4["market_share_sum"]

total_market_share = df_4[["operation","r_greater_exante","year","month","market_share_sum_diff","market_size"]].drop_duplicates()

        

In [133]:
total_market_share.groupby("operation")["market_share_sum_diff"].mean()*100

operation
Chain_affiliated   -0.134044
Independent         0.128308
Name: market_share_sum_diff, dtype: float64

In [134]:
df.groupby("operation")["ps_diff"].sum()/1000000

operation
Chain_affiliated    3.811157
Independent         5.145205
Name: ps_diff, dtype: float64

In [268]:
df.groupby(["r_greater_exante","operation"])["ps_diff"].sum()/1000000

r_greater_exante  operation       
False             Chain_affiliated    18.729764
                  Independent          6.735477
True              Chain_affiliated   -14.255042
                  Independent         -1.294047
Name: ps_diff, dtype: float64

In [269]:
df.groupby("operation")["revenue_diff"].sum()/1000000

operation
Chain_affiliated    1.403200
Independent         9.060572
Name: revenue_diff, dtype: float64

In [270]:
df.groupby(["r_greater_exante","operation"])["revenue_diff"].sum()/1000000

r_greater_exante  operation       
False             Chain_affiliated    48.649934
                  Independent         15.540101
True              Chain_affiliated   -47.246734
                  Independent         -6.479528
Name: revenue_diff, dtype: float64

In [271]:
df.groupby("operation")["cost_diff"].sum()/1000000

operation
Chain_affiliated   -3.071523
Independent         3.619142
Name: cost_diff, dtype: float64

In [272]:
df.groupby(["r_greater_exante","operation"])["cost_diff"].sum()/1000000

r_greater_exante  operation       
False             Chain_affiliated    29.920170
                  Independent          8.804624
True              Chain_affiliated   -32.991693
                  Independent         -5.185482
Name: cost_diff, dtype: float64

In [273]:
df.groupby(["operation","r_greater_exante"]).size()

operation         r_greater_exante
Chain_affiliated  False               2499
                  True                2329
Independent       False                385
                  True                 384
dtype: int64

In [274]:
df.groupby(["r_greater_exante"]).size()

r_greater_exante
False    2884
True     2713
dtype: int64